# Prepare Emotion detection model from classification model



# Load and prepare dataset


## Load video 

In [13]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [14]:
%cd /content

/content


In [15]:

!pip install opencv-contrib-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Create dirctories

In [16]:
import shutil , os

if os.path.exists('data/rowImg'):
  shutil.rmtree('data/rowImg')

In [17]:
dirs = [
    'data',
    'data/rowVid',
    'data/classification',
    'data/rowImg']

[os.makedirs(p) if not os.path.exists(p) else ''  for p in dirs]


[None, None, None, None]

In [18]:
%cp -r /gdrive/MyDrive/DataEnginnering/* /content/data/rowVid

In [19]:
import numpy as np
import cv2 as cv
import os
trial, count = 10, 0

for video in os.listdir('data/rowVid'):
  # if trial>count:
    if video.endswith('mp4') or video.endswith('avi'):
      cap = cv.VideoCapture('/content/data/rowVid/'+video)
      c=1
      success,image = cap.read()
      while success:
        if c%100==0:
          # resize image into input shape
          # to be implement
          cv.imwrite(f'/content/data/rowImg/'+ str(c) + '-' + video[:-4] + '.png', image)
          success,image = cap.read()
          print('Read a new frame: ', success)
        
        c= c+1
    
      cap.release()
  # count = count+1    

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

KeyboardInterrupt: ignored

In [ ]:
# import numpy as np
# import cv2 as cv

# for video in os.listdir('data/rowVid'):
#   if video.endswith('mp4') or video.endswith('avi'):
#     cap = cv.VideoCapture('/content/data/rowVid/'+video)
#     c=1
#     success,image = cap.read()
#     while success:
#       if c%100==0:
#         cv.imwrite(f'/content/data/rowImg/'+ str(c) + '-' + video[:-4] + '.png', image)     # save frame as JPEG file      
#         success,image = cap.read()
#         print('Read a new frame: ', success)
      #  x
#       c= c+1
   
#     cap.release()

# Load classifier model

In [29]:
%mkdir model

mkdir: cannot create directory ‘model’: File exists


In [30]:
!wget -O model/emotionModel.hdf5 https://github.com/Furkan-Gulsen/face-classification/raw/main/models/emotionModel.hdf5
# https://github.com/Furkan-Gulsen/face-classification/blob/main/emotionRecognition.py implementatino code

--2023-05-08 09:04:18--  https://github.com/Furkan-Gulsen/face-classification/raw/main/models/emotionModel.hdf5
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Furkan-Gulsen/face-classification/main/models/emotionModel.hdf5 [following]
--2023-05-08 09:04:18--  https://raw.githubusercontent.com/Furkan-Gulsen/face-classification/main/models/emotionModel.hdf5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 872856 (852K) [application/octet-stream]
Saving to: ‘model/emotionModel.hdf5’

model/emotionModel. 100%[===================>] 852.40K  --.-KB/s    in 0.05s   

2023-05-08 09:04:18 (18.4 MB/s) - ‘

In [31]:
from keras.models import load_model
import numpy as np
import cv2

In [32]:
emotionModelPath = 'model/emotionModel.hdf5'  # fer2013_mini_XCEPTION.110-0.65
emotionClassifier = load_model(emotionModelPath, compile=False)
emotionTargetSize = emotionClassifier.input_shape[1:3]

In [35]:
def frame_to_prid(x,y,h,w,img):
        grayFrame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        grayFace = grayFrame[y:y + h, x:x + w]
        
        grayFace = cv2.resize(grayFace, (emotionTargetSize))
    
        grayFace = grayFace.astype('float32')
        grayFace = grayFace / 255.0
        grayFace = (grayFace - 0.5) * 2.0
        grayFace = np.expand_dims(grayFace, 0)
        grayFace = np.expand_dims(grayFace, -1)
        emotion_prediction = emotionClassifier.predict(grayFace)
        emotion_probability = np.max(emotion_prediction)
        # print(emotion_prediction)
        # print(emotion_probability)
        emotion_label_arg = np.argmax(emotion_prediction)
        print(emotions[emotion_label_arg]['emotion'])
        print(emotion_label_arg)
        return str(emotion_label_arg)

In [34]:
emotions = {
    0: {
        "emotion": "Angry",
        "color": (193, 69, 42)
    },
    1: {
        "emotion": "Disgust",
        "color": (164, 175, 49)
    },
    2: {
        "emotion": "Fear",
        "color": (40, 52, 155)
    },
    3: {
        "emotion": "Happy",
        "color": (23, 164, 28)
    },
    4: {
        "emotion": "Sad",
        "color": (164, 93, 23)
    },
    5: {
        "emotion": "Suprise",
        "color": (218, 229, 97)
    },
    6: {
        "emotion": "Neutral",
        "color": (108, 72, 200)
    }
}


# Extract Faces from rowImage using mtcnn

In [36]:
!pip install pybboxes mtcnn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
from mtcnn import MTCNN
import cv2
from pybboxes import BoundingBox
import os

In [ ]:
for fn in os.listdir('/content/data/rowImg/'):
  if os.path.exists(f'/content/data/rowImg/'+fn) and (fn.endswith('.png') or fn.endswith('.jpg')):
    print(f'/content/data/rowImg/'+fn)
    img = cv2.cvtColor(cv2.imread(f'/content/data/rowImg/'+fn), cv2.COLOR_BGR2RGB)
    detector = MTCNN()
    # print(detector.detect_faces(img))
    for face in detector.detect_faces(img):
      try:
        # create bbox
        a = face['box']
        coco_bbox = BoundingBox.from_coco(*a)
        coco_bbox.image_size = (640, 480)
        yolo_bbox = coco_bbox.to_yolo()

        #get prid
        prid = frame_to_prid(a[0], a[1], a[2], a[3], img) 

        # save yolo bbox
        f = open(f'/content/data/rowImg/'+fn[:-4]+'.txt', "w")
        f.write(f''+ prid +' '+ str(yolo_bbox.values[0]) + ' '+ str(yolo_bbox.values[1]) + ' '+ str(yolo_bbox.values[2]) + ' '+ str(yolo_bbox.values[3]) + ' ')
        f.close()
      except:
        continue
      # save croped faces to new folder new file
      

/content/data/rowImg/262800-carpoolKaraoke.png
1/1 [==============================] - 0s 19ms/step
Sad
4
1/1 [==============================] - 0s 19ms/step
Angry
0
/content/data/rowImg/195500-carpoolKaraoke.png
1/1 [==============================] - 0s 18ms/step
Sad
4
1/1 [==============================] - 0s 18ms/step
Happy
3
/content/data/rowImg/137700-carpoolKaraoke.png
1/1 [==============================] - 0s 27ms/step
Fear
2
/content/data/rowImg/291000-carpoolKaraoke.png
1/1 [==============================] - 0s 23ms/step
Happy
3
/content/data/rowImg/74300-carpoolKaraoke.png
1/1 [==============================] - 0s 21ms/step
Sad
4
1/1 [==============================] - 0s 19ms/step
Happy
3
1/1 [==============================] - 0s 21ms/step
Happy
3
/content/data/rowImg/378900-carpoolKaraoke.png
1/1 [==============================] - 0s 21ms/step
Neutral
6
1/1 [==============================] - 0s 26ms/step
Sad
4
/content/data/rowImg/193500-carpoolKaraoke.png
1/1 [==============

[link text](https://)## Extract frames(images from video)

In [ ]:
!cat /content/data/rowImg/14400-carpoolKaraoke.txt